In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow import keras
import seaborn as sbn

In [2]:
# import data TODO: Change the file path to the dataset this question needed
df = pd.read_csv('..\\source_data\\test.csv')
df

,Date,gasoline,crude_futures,gdp
0,1-Jan-01,1.406,27.95,12999.56990
1,8-Jan-01,1.425,30.05,12293.34053
2,15-Jan-01,1.474,32.19,12708.81038
3,22-Jan-01,1.471,29.77,12287.22655
4,29-Jan-01,1.460,31.19,13353.26284
...,...,...,...,...
412,24-Nov-08,1.892,54.43,15451.90316
413,1-Dec-08,1.811,40.81,15362.37041
414,8-Dec-08,1.699,46.28,14770.91456
415,15-Dec-08,1.659,33.87,15966.69919


# Creating independent and dependent variables 

In [3]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df['of_chg_{}'.format(i)] = df['crude_futures'].pct_change(periods = i)
    
# calculate dependent variable
df['gas_chg'] = df['gasoline'].pct_change()

# eliminate the empty rows
df = df[11:]
df

,Date,gasoline,crude_futures,gdp,of_chg_1,of_chg_2,of_chg_3,of_chg_5,of_chg_10,gas_chg
11,19-Mar-01,1.404,27.30,11982.22891,0.020942,-0.025348,-0.019397,-0.063786,-0.091514,-0.005666
12,26-Mar-01,1.404,26.29,13936.69786,-0.036996,-0.016829,-0.061407,-0.094697,-0.183287,0.000000
13,2-Apr-01,1.442,27.06,12809.32454,0.029289,-0.008791,0.011967,-0.028017,-0.091031,0.027066
14,9-Apr-01,1.500,28.25,13474.20064,0.043976,0.074553,0.034799,0.008568,-0.094261,0.040222
15,16-Apr-01,1.571,27.28,13236.22196,-0.034336,0.008130,0.037657,0.020194,-0.120851,0.047333
...,...,...,...,...,...,...,...,...,...,...
412,24-Nov-08,1.892,54.43,15451.90316,0.090126,-0.045757,-0.108290,-0.151520,-0.479388,-0.086873
413,1-Dec-08,1.811,40.81,15362.37041,-0.250230,-0.182656,-0.284537,-0.398171,-0.618206,-0.042812
414,8-Dec-08,1.699,46.28,14770.91456,0.134036,-0.149734,-0.073102,-0.241809,-0.507030,-0.061844
415,15-Dec-08,1.659,33.87,15966.69919,-0.268150,-0.170056,-0.377733,-0.406206,-0.564093,-0.023543


# Seperating training, validation, and test datasets

In [4]:
# create list of independent variable's name TODO: add the appropriate variable names to the list
ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['gdp']
num_var =len(ind_var_name) # put number of variables here

# extract the values to X
X = df[['crude_futures'] + ind_var_name].values
X

array([[ 2.73000000e+01,  2.09424084e-02, -2.53480900e-02, ...,
        -6.37860082e-02, -9.15141431e-02,  1.19822289e+04],
       [ 2.62900000e+01, -3.69963370e-02, -1.68287210e-02, ...,
        -9.46969697e-02, -1.83286735e-01,  1.39366979e+04],
       [ 2.70600000e+01,  2.92887029e-02, -8.79120879e-03, ...,
        -2.80172414e-02, -9.10312395e-02,  1.28093245e+04],
       ...,
       [ 4.62800000e+01,  1.34035776e-01, -1.49733603e-01, ...,
        -2.41808650e-01, -5.07030251e-01,  1.47709146e+04],
       [ 3.38700000e+01, -2.68150389e-01, -1.70056359e-01, ...,
        -4.06206171e-01, -5.64092664e-01,  1.59666992e+04],
       [ 3.77100000e+01,  1.13374668e-01, -1.85177182e-01, ...,
        -2.44742640e-01, -4.75156576e-01,  1.53330495e+04]])

In [5]:
# create y for regressor, y_c for classifier
y = df['gas_chg']
y_c = (df['gas_chg'] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# check correctness of X and y
X.shape, y.shape, y_c.shape

((406, 7), (406,), (406,))

In [6]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train shape:	 (284, 7)
X_test shape:	 (61, 7)
X_val shape:	 (61, 7)
y_train shape:	 (284,)
y_val shape:	 (61,)
y_test shape:	 (61,)


In [7]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size = test_ratio, random_state = 42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c shape: (284, 7)
X_test_c shape:	 (61, 7)
X_val_c shape:	 (61, 7)
y_train_c shape: (284,)
y_val_c shape:	 (61,)
y_test_c shape:	 (61,)


# Principal Component Analysis (PCA)

In [8]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

X_train shape:	 (284, 7)
X_val shape:	 (61, 7)


In [9]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

X_train_c shape: (284, 7)
X_val_c shape:	 (61, 7)


In [10]:
# PCA for regressor
# fit on training set
scaler = StandardScaler()
scaler.fit(X_train)

# apply transform to training and test set
X_train_pca = scaler.transform(X_train)
X_test_pca = scaler.transform(X_test)
X_val_pca = scaler.transform(X_val)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_pca)

# apply transform to training and test set
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
X_val_pca = pca.transform(X_val_pca)
print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train_pca shape:	 (284, 6)
X_test_pca shape:	 (61, 6)
X_val_pca shape:	 (61, 6)
y_train shape:	 (284,)
y_val shape:	 (61,)
y_test shape:	 (61,)


In [11]:
# PCA for classifier
# fit on training set_c
scaler = StandardScaler()
scaler.fit(X_train_c)

# apply transform to training and test set
X_train_c_pca = scaler.transform(X_train_c)
X_test_c_pca = scaler.transform(X_test_c)
X_val_c_pca = scaler.transform(X_val_c)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_c_pca)

# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c_pca)
X_test_c_pca = pca.transform(X_test_c_pca)
X_val_c_pca = pca.transform(X_val_c_pca)
print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c_pca shape:	 (284, 6)
X_test_c_pca shape:	 (61, 6)
X_val_c_pca shape:	 (61, 6)
y_train_c shape: (284,)
y_val_c shape:	 (61,)
y_test_c shape:	 (61,)


# Multiple linear regression

In [12]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)
print('score_multiple_reg:', "%.4f"%score_multiple_reg)

score_multiple_reg: 0.4883


# Multiple polynomial regression

In [13]:
#to prove this is the correct polynomial regression, set degree to 1
poly = PolynomialFeatures(1)
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.4883117059552289

In [14]:
# use val to find the appropriate degree
mpr = dict()
for i in range(2,6):
    degree=i
    poly = PolynomialFeatures(degree)
    # polynomial transform
    X_train_poly = poly.fit_transform(X_train_pca, y_train)
    X_val_poly = poly.fit_transform(X_val_pca,y_val)
    X_test_poly = poly.fit_transform(X_test_pca,y_test)

    # fit to linear regression
    polyreg = LinearRegression().fit(X_train_poly, y_train)

    # predict using liear regression
    score_poly_reg = polyreg.score(X_val_poly, y_val)
    print('score_poly_reg, degree',i,':', "%.4f"%score_poly_reg)
    mpr[f'{i}'] = score_poly_reg
    
max_degree = max(mpr, key = mpr.get)
print('max degree:', max_degree)

score_poly_reg, degree 2 : 0.1461
score_poly_reg, degree 3 : -0.4796
score_poly_reg, degree 4 : -99.4928
score_poly_reg, degree 5 : -5031.6696
max degree: 2


In [15]:
poly = PolynomialFeatures(int(max_degree))
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.41955643641167384

# Random forest classifier

In [16]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[f'{i}'] = score_clf
    #print('score_clf_',i,":", "%.4f"%score_clf)
max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

max depth: 8


In [17]:
# choose max_depth = 4
clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f"%score_rf_clf)

score_rf_clf: 0.7213


# Random forest regressor

In [18]:
clf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = clf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f"%score_rf_reg)

score_rf_reg: 0.2993


# Logistic regression

In [19]:
logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_model.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f"%score_logistic)

score_logistic: 0.7705


# Gradient boosting classifier

In [20]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

0.7213114754098361

# Gradient boosting regressor 

In [21]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

0.2988639984077174

# XGBoost classifier

In [22]:
# TODO:这个的参数是可以调整的 https://github.com/EpistasisLab/tpot/issues/1139 
xgb_clf = XGBClassifier().fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

[14:29:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\lemon\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7213114754098361

# XGBoost regressor

In [23]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

0.21998049720317925

# AdaBoost classifier

In [24]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

0.7868852459016393

# AdaBoost regressor

In [25]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

0.08710383101875008

# Support vector classification

In [26]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

0.7213114754098361
0.6557377049180327
0.6721311475409836


# Gaussian Naive Bayes

In [27]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

0.7049180327868853


# Neuro networks

In [28]:
# TODO: 这个地方的input_dim是不是就是X一共有多少variable
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

Epoch 1/50
9/9 [==============================] - 1s 108ms/step - loss: 0.7718 - accuracy: 0.3897 - val_loss: 0.7835 - val_accuracy: 0.3607
Epoch 2/50
9/9 [==============================] - 0s 4ms/step - loss: 0.7489 - accuracy: 0.4467 - val_loss: 0.7646 - val_accuracy: 0.3607
Epoch 3/50
9/9 [==============================] - 0s 4ms/step - loss: 0.7494 - accuracy: 0.4537 - val_loss: 0.7460 - val_accuracy: 0.4262
Epoch 4/50
9/9 [==============================] - 0s 4ms/step - loss: 0.7120 - accuracy: 0.4993 - val_loss: 0.7295 - val_accuracy: 0.4426
Epoch 5/50
9/9 [==============================] - 0s 4ms/step - loss: 0.6886 - accuracy: 0.5550 - val_loss: 0.7121 - val_accuracy: 0.4754
Epoch 6/50
9/9 [==============================] - 0s 4ms/step - loss: 0.6812 - accuracy: 0.6028 - val_loss: 0.6950 - val_accuracy: 0.4918
Epoch 7/50
9/9 [==============================] - 0s 4ms/step - loss: 0.6648 - accuracy: 0.6070 - val_loss: 0.6798 - val_accuracy: 0.5246
Epoch 8/50
9/9 [================

In [29]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

2/2 [==============================] - 0s 1ms/step - loss: 0.5014 - accuracy: 0.6885
0.688524603843689


# Summary

In [30]:
reg = {
    'Multiple linear regression': score_multiple_reg,
    'Multiple polynomial regression': score_poly_reg,
    'Logistic regression': score_logistic,
    'Random forest regressor': score_rf_reg,
    'Gradient boosting regressor': score_gb_reg,
    'XGBoost regressor': score_xgb_reg,
    'AdaBoost regressor': score_adb_reg
}
reg_df = pd.DataFrame.from_dict(reg, orient='index', columns = ['Score'])
reg_df.index.name = 'Regression models'
reg_df.sort_values(by=['Score'], ascending=False)

,Score
Regression models,
Logistic regression,0.770492
Multiple linear regression,0.488312
Multiple polynomial regression,0.419556
Random forest regressor,0.299276
Gradient boosting regressor,0.298864
XGBoost regressor,0.219980
AdaBoost regressor,0.087104


In [31]:
clf = {
    'Random forest classifier': score_rf_clf,
    'Gradient boosting classifier': score_gb_clf,
    'XGBoost classifier': score_xgb_clf,
    'AdaBoost classifier': score_adb_clf,
    'Gaussian naive Bayes': score_gnb, 
    'SVC linear': score_svc_linear,
    'SVC polynomial': score_svc_poly, 
    'SVC RBF': score_svc_rbf,
    'Neuro network' : score_neuro
}
clf_df = pd.DataFrame.from_dict(clf, orient='index',columns = ['Score'])
clf_df.index.name = 'Classifier models'
clf_df.sort_values(by=['Score'], ascending=False)

,Score
Classifier models,
AdaBoost classifier,0.786885
Random forest classifier,0.721311
Gradient boosting classifier,0.721311
XGBoost classifier,0.721311
SVC linear,0.721311
Gaussian naive Bayes,0.704918
Neuro network,0.688525
SVC RBF,0.672131
SVC polynomial,0.655738
